# Introducción
Explorar el formato de los datos del metro

# Config


In [20]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime
from google.transit import gtfs_realtime_pb2
import time
from realtime.vehicles import load_positions_metro
from static.stations import load_metro_stations

# 
METRO_VEHICLES_URL = "https://ctb-gtfs-rt.s3.eu-south-2.amazonaws.com/metro-bilbao-vehicle-positions.pb"
METRO_STATIONS_URL ="https://www.metrobilbao.eus/wp-content/uploads/open-data/MB%20GTFS.zip"


# Load data

In [ ]:
df_metro = load_positions_metro(METRO_VEHICLES_URL)


,vehicle_id,lat,lon,timestamp,mode
0,None,43.350449,-3.00935,2025-11-30 23:41:36,metro


In [21]:
df_stations_metro = load_metro_stations(METRO_STATIONS_URL)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

# Data management (Code)

# Plots

In [3]:
# Plots and visualizations

# Save results

In [5]:
#Save results and figures